In [ ]:
%run ParsingMethods.ipynb

timestamps = {}
all_contributions = {}
all_folders = []

all_contributions, timestamps, all_folders = parseCSV('contributions.csv')

In [ ]:
# Create a map of shortened questions to nodes so we can cluster and then refer back
# For now, limit this to just one hw assignment
folder = "hw4"

to_cluster = {}
for node in all_contributions.values():
    if folder in node.question_folders:
        to_cluster[node.shortened_question] = node

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score

from xml.dom import minidom
keys = list(to_cluster.keys())

print(len(keys))

# parse an xml file by name
mydoc = minidom.parse('Posts.xml')

SEitems = mydoc.getElementsByTagName('row')

SEitems_questions = []
for item in SEitems:
    SEitems_questions.append(item.attributes['Body'].value)
    
SEitems_questions = SEitems_questions[1:6000]

vectorizer = TfidfVectorizer(stop_words='english')
X_orig = vectorizer.fit(SEitems_questions)
X = vectorizer.transform(keys)

# Number of clusters per homework
true_k = 100
model = KMeans(n_clusters=true_k, init='k-means++', max_iter=100, n_init=10)
model.fit(X)

# print("\n")
# print("Prediction")




In [ ]:
Y = vectorizer.transform(["Should we fail for empty queues for delete and contains too?"])
prediction = model.predict(Y)

In [ ]:
final_clusters = {}
cluster_location = {}
for i in range(len(keys)):
    label = model.labels_[i]
    final_clusters.setdefault(label, []).append(to_cluster[keys[i]])
    cluster_location[to_cluster[keys[i]]] = label
    
# print(raw_references)
    
# print(final_clusters)

In [ ]:
from unionfind import unionfind

length = len(list(final_clusters.keys()))
u = unionfind(length)

# Hard coded for not follow ups right now
# Getting some errors -- I think with questions that are outside this hw assignment
for key in cluster_location.keys():
    key_cluster_number = cluster_location[key]
    print(key.question_number)
    print(key.references)
    for ref in key.references:
        if not ref == -1:
            try:
                num_to_unite = cluster_location[all_contributions[(str(ref), 0)]]
                u.unite(key_cluster_number, num_to_unite)
            except:
                print("err")

print(u.groups())